In [55]:
def read_link_data(knows_fname, knows_target_fname, likes_fname, lived_fname):
    people_dict = dict()
    people_id_dict = dict()
    
    interest_dict = dict()
    interest_id_dict = dict()
    
    places_dict = dict()
    places_id_dict = dict()
    
    def dict_man(constant_dict, id_dict):
        def str2id(cstr):
            if cstr in constant_dict:
                return constant_dict[cstr]
            id = len(constant_dict)
            constant_dict[cstr] = id
            id_dict[id] = cstr
            return id
        return str2id
    
    people_id = dict_man(people_dict, people_id_dict)
    interest_id = dict_man(interest_dict, interest_id_dict)
    place_id = dict_man(places_dict, places_id_dict)
            
    knows_rel = dict()
    with open(knows_fname) as knows_file:
        for line in knows_file:
            line = line.strip()
            if not line: continue
            line = line.split()
            current_tuple = (people_id(line[0]), people_id(line[1]))
            knows_rel[current_tuple] = (True, 1.0)
            
    with open(knows_target_fname) as knows_file:
        for line in knows_file:
            line = line.strip()
            if not line: continue
            line = line.split(',')
            current_tuple = (people_id(line[0][1:-1]), people_id(line[1][1:-1]))
            knows_rel[current_tuple] = (True, float(line[2]))            
            
            
    likes_rel = dict()
    with open(likes_fname) as likes_file:
        for line in likes_file:
            line = line.strip()
            if not line: continue
            line = line.split()            
            current_tuple = (people_id(line[0]), interest_id(' '.join(line[1:-1])))
            likes_rel[current_tuple] = (True, float(line[-1]))
            
    lived_rel = dict()
    with open(lived_fname) as lived_file:
        for line in lived_file:
            line = line.strip()
            if not line: continue
            line = line.split()            
            current_tuple = (people_id(line[0]), place_id(' '.join(line[1:])))
            lived_rel[current_tuple] = (True, 1.0)
            
            

            
    for person in people_id_dict:
        for interest in interest_id_dict:
            current_tuple = (person, interest)
            if not current_tuple in likes_rel:
                likes_rel[current_tuple] = (True, 0.0)
                
    for person in people_id_dict:
        for place in places_id_dict:
            current_tuple = (person, place)
            if not current_tuple in lived_rel:
                lived_rel[current_tuple] = (True, 0.0)
                
    return knows_rel, likes_rel, lived_rel, people_id_dict, interest_id_dict, places_id_dict

In [125]:
def get_obj(knows_rel, likes_rel, 
            lived_rel, people, 
            interests, places):
    
    obj = 0.0
    
    # 20:  Lived(P1,L) & Lived(P2,L) & P1!=P2   -> Knows(P1,P2)
    ground_rules = [(lived_rel[(A, C)][1],
                     lived_rel[(B, C)][1],
                     float(A!=B),
                     knows_rel[(A, B)][1])
                    for A in people for B in people 
                    for C in places if A!=B]

    weight = 20
    for P1, P2, P3, P4 in ground_rules:
        d = max(P1+P2-1, 0)
        d = max(d+P3-1, 0)
        d = min(1-d + P4, 1)
        obj += weight * d

    # 5:  Lived(P1,L1) & Lived(P2,L2) & P1!=P2 & L1!=L2  -> !Knows(P1,P2)
    ground_rules = [(lived_rel[(A, C)][1],
                     lived_rel[(B, D)][1],
                     float(A!=B),
                     float(C!=D),                     
                     knows_rel[(A, B)][1])
                    for A in people for B in people 
                    for C in places for D in places
                    if (A!=B)]

    weight = 5
    for P1, P2, P3, P4, P5 in ground_rules:
        d = max(P1+P2-1, 0)
        d = max(d+P3-1, 0)
        d = max(d+P4-1, 0)
        d = min(1-d + 1-P5, 1)
        obj += weight * d    

    # 10:  Likes(P1,L) & Likes(P2,L) & P1!=P2  -> Knows(P1,P2)
    ground_rules = [(likes_rel[(A, C)][1],
                     likes_rel[(B, C)][1],
                     float(A!=B),                     
                     knows_rel[(A, B)][1])
                    for A in people for B in people 
                    for C in interests
                    if A!=B]

    weight = 10
    for P1, P2, P3, P4 in ground_rules:
        d = max(P1+P2-1, 0)
        d = max(d+P3-1, 0)
        d = min(1-d + P4, 1)
        obj += weight * d   

    # 5:   Knows(P1,P2) & Knows(P2,P3) & P1!=P3 -> Knows(P1,P3)
    ground_rules = [(knows_rel[(A, B)][1],
                     knows_rel[(B, C)][1],
                     float(A!=C),                     
                     knows_rel[(A, C)][1])
                    for A in people 
                    for B in people 
                    for C in people
                    if (A!=B and B!=C and A!=C)]

    weight = 5
    for P1, P2, P3, P4 in ground_rules:
        d = max(P1+P2-1, 0)
        d = max(d+P3-1, 0)
        d = min(1-d + P4, 1)
        obj += weight * d  

    # 10000: Knows(P1,P2) -> Knows(P2,P1)
    ground_rules = [(knows_rel[(A, B)][1],
                     knows_rel[(B, A)][1])
                    for A in people 
                    for B in people 
                    if A!=B]
    weight = 10000
    for P1, P2 in ground_rules:
        d = min(1-P1 + P2, 1)
        obj += weight * d


    # 5:  !Knows(P1,P2)
    ground_rules = [knows_rel[(A, B)][1]
                    for A in people 
                    for B in people 
                    if A!=B]
    weight = 5
    for P in ground_rules:
        d = 1-P
        obj += weight * d
                
    return obj

In [126]:
data = read_link_data('./data/knows_obs.txt',
                                    './knows_infer.csv',
                                    './data/likes_obs.txt',
                                    './data/lived_obs.txt')
print(get_obj(*data))

586188.0


In [127]:
data = read_link_data('./data/knows_obs.txt',
                                    '/home/behrouz/temp/test_psl_cli/output/KNOWS.csv',
                                    './data/likes_obs.txt',
                                    './data/lived_obs.txt')
print(get_obj(*data))

586097.3492441818
